# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [65]:
# Libraries
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [66]:
# your code here

admissions = pd.read_csv('data/Admission_Predict.csv')
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [67]:
# your code here

admissions_clean = admissions.rename(columns=lambda x: x.strip())

Let's evaluate the dataset by looking at the `head` function.

In [68]:
# your code here
admissions_clean.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [69]:
# your code here
admissions_clean.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [70]:
# your code here
admissions_index = admissions_clean.set_index('Serial No.')
admissions_index

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [71]:
# your code here
#the values would have to be unique

def check_duplicates(list_name):
    ''' Check if given list contains any duplicates '''    
    for elem in list_name:
        if list_name.count(elem) > 1:
            return True and print("there are non unique values")
    return False

In [72]:
list1 = admissions_index['GRE Score'].tolist()
check_duplicates(list1)

there are non unique values


In [73]:
list2 = admissions_index['CGPA'].tolist()
check_duplicates(list2)

there are non unique values


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [74]:
# your code here

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [75]:
# your code here
admissions_index[(admissions_index['CGPA'] > 9) & (admissions_index['Research'] == 1)].count()

GRE Score            101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
CGPA                 101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [76]:
# your code here
a = admissions_index[(admissions_index['CGPA'] > 9) & (admissions_index['SOP'] < 3.5)]
a

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
29,338,118,4,3.0,4.5,9.40,1,0.91
63,327,114,3,3.0,3.0,9.02,0,0.61
141,326,114,3,3.0,3.0,9.11,1,0.83
218,324,111,4,3.0,3.0,9.01,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84


In [77]:
b = a['Chance of Admit'].mean()
print('The mean chance of admitting a graduate with a CGPA higher than 9 and a SOP lower than 3.5 if of ', b)

The mean chance of admitting a graduate with a CGPA higher than 9 and a SOP lower than 3.5 if of  0.8019999999999999


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [78]:
#def standardize(col):
"""
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
"""
    
    # your code here
def standardize(col):
    
    return ( (col - col.mean()) / col.std() )

#testing code
standardize(admissions_index['CGPA'])

Serial No.
1      1.747900
2     -0.991211
3      0.121034
4     -0.642597
5      1.233279
         ...   
381    0.735259
382    0.851463
383    1.415886
384    0.303641
385    1.764500
Name: CGPA, Length: 385, dtype: float64

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [79]:
# your code here
admissions_index['CGPA_std'] = standardize(admissions_index['CGPA'])
admissions_index['GRE_std'] = standardize(admissions_index['GRE Score'])
admissions_index['LOR_std'] = standardize(admissions_index['LOR'])
admissions_index.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
Serial No.,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647
2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740
3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166
4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713
5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [80]:
# Libraries
from random import choices

In [81]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions_index.shape[0])
decision_choice

['LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'LOR_st

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [82]:
# your code here
#lookup function: https://www.w3resource.com/pandas/dataframe/dataframe-lookup.php

admissions_index['deciding_column'] = admissions_index.lookup(admissions_index.index, decision_choice)
admissions_index

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
Serial No.,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.191647
2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,-0.991211
3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704
4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597
5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,1.147798
...,...,...,...,...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82,0.735259,0.628727,0.076740,0.735259
382,325,107,3,3.0,3.5,9.11,1,0.84,0.851463,0.715239,0.076740,0.851463
383,330,116,4,5.0,4.5,9.45,1,0.91,1.415886,1.147798,1.191647,1.191647


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [83]:
# your code here

deciding_value = []

for x in admissions_index['deciding_column']:
    if x > 0.8:
        deciding_value.append(1)
    else: 
        deciding_value.append(0)
deciding_value

[1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,


In [84]:
admissions_index['decision'] = deciding_value
admissions_index.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
Serial No.,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.191647,1
2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0
3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704,0
4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0
5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,1.147798,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [85]:
c = admissions_index['decision'].value_counts()
print("Based on the decision column,", c[1], " applicants will be accepted")

Based on the decision column, 95  applicants will be accepted


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [86]:
# your code here
#remove all non character symbols, using regex: re.sub("[^a-zA-Z]+", " ")
#replace spaces with underscore: .replace(' ','_')
#upper case to lower: .lower()

import re
d = admissions_index #safeguarding the original format
d.columns = ["".join(re.sub('[^A-Za-z0-9]+', ' ', x.lower())).replace(' ','_') for x in list(d.columns)]
d.head()

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
Serial No.,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.191647,1
2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0
3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704,0
4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0
5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,1.147798,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [89]:
# your code here

d.loc[d['university_rating'] >= 4,'adjusted_gre'] = d['gre_score']+10
d.head()

,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
Serial No.,,,,,,,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92,1.747900,1.753381,1.191647,1.191647,1,347.0
2,316,104,3,3.0,3.5,8.00,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0,NaN
3,322,110,3,3.5,2.5,8.67,1,0.80,0.121034,0.455704,-1.038166,0.455704,0,NaN
4,314,103,2,2.0,3.0,8.21,0,0.65,-0.642597,-0.236391,-0.480713,-0.642597,0,NaN
5,330,115,5,4.5,3.0,9.34,1,0.90,1.233279,1.147798,-0.480713,1.147798,1,340.0


In [91]:
admissions_bins = pd.cut(d['adjusted_gre'],4)
admissions_bins

Serial No.
1      (337.5, 350.0]
2                 NaN
3                 NaN
4                 NaN
5      (337.5, 350.0]
            ...      
381               NaN
382               NaN
383    (337.5, 350.0]
384               NaN
385    (337.5, 350.0]
Name: adjusted_gre, Length: 385, dtype: category
Categories (4, interval[float64]): [(299.95, 312.5] < (312.5, 325.0] < (325.0, 337.5] < (337.5, 350.0]]